<a href="https://colab.research.google.com/github/ssungni/Algorithm/blob/main/pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
# torch라고 하는 프로그래밍 언어가 먼저 나옴,
# 루아(Lua)라고 구현된 토치 라이브러리에 파이썬을 적용해서 API -> pytorch

from torch import nn
from torch.utils.data import DataLoader
# "torch.utils.data.DataLoader"과 같이 매번 써주는 게 쉽지 않으므로
DataLoader # 나는 DataLoader로 쓸거야!!

from torchvision import datasets # 파이토치에서 사용하는 컴퓨터 비전들이 굉장히 많이 구현되어 있음
# 이러한 datasets를 관리하는 transforms에 Tensor
from torchvision.transforms import ToTensor, Lambda, Compose

import matplotlib.pyplot as plt

In [3]:
# datasets -> FashionMNIST -> training data, test data를 불러오는 과정

# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True, # FashionMNIST에는 train 데이터, test 데이터가 있으며 그 중 train (o)
    download=True, # if 다운로드를 안 했으면 다운로드를 하여라
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26421880/26421880 [00:02<00:00, 11686293.00it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 207459.26it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3893346.95it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4807899.58it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [10]:
# dataloader => 데이터로더가 데이터를 64개씩 묶어서 처리하도록

batch_size = 64

train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader: # X: 데이터(ex) 이미지), y: 데이터에 대한 레이블(ex) 고양이, 강아지)
  print("Shape of X (N, C, H, W): ", X.shape) # 4차원 N: Number(수), C: Channel(채널의 길이), H: High(높이), W: Width(가로)
  print("Shape of y : ", y.shape, y.dtype) # 1차원
  break

Shape of X (N, C, H, W):  torch.Size([64, 1, 28, 28])
Shape of y :  torch.Size([64]) torch.int64


In [11]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu" # cuda란, NVIDIA에서 개발한 GPU 개발 툴
print("Using {} device".format(device))

Using cpu device


In [13]:
# creating model
class NeuralNetwork(nn.Module):
# nn.Module라는 부모 클래스 or 슈퍼 클래스로부터 상속을 받아 NeuralNetwork를 만들거임
# nn.Module을 받는 다는 건 이미 초기화도 되어 있고, 정의가 되어 있지만
# 우리가 재 정의할 수 있다!

    def __init__(self): # 초기화
        super(NeuralNetwork, self).__init__() # super 클래스를 먼저 초기화
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential( # Sequential하다는 것은 Linear, ReLU 를 다음과 같이 반복하는
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x): # 방법 재정의, 실행은 train 함수에서
        x = self.flatten(x) # (flatten): Flatten(start_dim=1, end_dim=-1)
        logits = self.linear_relu_stack(x) # Linear, ReLU
        return logits

model = NeuralNetwork().to(device)
print(model)
# 64개를 모은 결과, 최종 feature가 10개이어야 함.

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [17]:
# 실행
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device) # 사전에 어디에 올릴지 정의하고 해야

        # Compute prediction error
        pred = model(X)  # model(X) -> forward 함수를 정의해라 => nn.module에 다 있음
        loss = loss_fn(pred, y) # 얼마나 비슷해? 얼마나 정확해?

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() # 어느 지점에서 어떻게 내려왔어

        if batch % 100 == 0: # 100번에 한 번씩 프린트를 해라! 얼마만큼 되었는
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [16]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [18]:
epochs = 5 # 100번에 한 번, 한 번 = 64회 그러므로 6400씩일 때
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

# Accuracy => 학습하지 않은 것에서 잘 나와야

Epoch 1
-------------------------------
loss: 2.297005  [    0/60000]
loss: 2.288083  [ 6400/60000]
loss: 2.274117  [12800/60000]
loss: 2.273103  [19200/60000]
loss: 2.260358  [25600/60000]
loss: 2.246846  [32000/60000]
loss: 2.252670  [38400/60000]
loss: 2.239069  [44800/60000]
loss: 2.220069  [51200/60000]
loss: 2.201971  [57600/60000]
Test Error: 
 Accuracy: 49.2%, Avg loss: 0.034576 

Epoch 2
-------------------------------
loss: 2.203372  [    0/60000]
loss: 2.178888  [ 6400/60000]
loss: 2.153715  [12800/60000]
loss: 2.180137  [19200/60000]
loss: 2.129426  [25600/60000]
loss: 2.117247  [32000/60000]
loss: 2.141595  [38400/60000]
loss: 2.112052  [44800/60000]
loss: 2.087985  [51200/60000]
loss: 2.050860  [57600/60000]
Test Error: 
 Accuracy: 51.6%, Avg loss: 0.032243 

Epoch 3
-------------------------------
loss: 2.052691  [    0/60000]
loss: 1.998588  [ 6400/60000]
loss: 1.958560  [12800/60000]
loss: 2.030573  [19200/60000]
loss: 1.910345  [25600/60000]
loss: 1.916396  [32000/600

## Build the Neural Network

In [19]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [20]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [22]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [26]:
X = torch.rand(1, 28, 28, device=device) # channel rgb면 3, 그러나 우리는 gray scale이므로 1임, (세로, 가로)
logits = model(X)
print(logits) # Softmax를 거치기 전이라 합이 1이 아님
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab) # Softmax를 거쳐서 합이 1임
y_pred = pred_probab.argmax(1) # 여기서 제일 큰 값은
print(f"Predicted class: {y_pred}") # 3번째

tensor([[0.0000, 0.0000, 0.0000, 0.0479, 0.0000, 0.0418, 0.0000, 0.0000, 0.0253,
         0.0262]], grad_fn=<ReluBackward0>)
tensor([[0.0986, 0.0986, 0.0986, 0.1034, 0.0986, 0.1028, 0.0986, 0.0986, 0.1011,
         0.1012]], grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([3])


In [27]:
# 순서 0. 입력
input_image = torch.rand(3,28,28) # 채널이 3개인 이미지
print(input_image.size())

torch.Size([3, 28, 28])


In [28]:
# 순서 1. flatten
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [29]:
# 순서 2. Linear
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [30]:
# 순서 0 -> 1 -> 2

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1470,  0.3908, -0.4323,  0.2342,  0.1481,  0.1198,  0.1688, -0.2064,
          0.0590, -0.3100,  0.1446,  0.1083,  0.1193, -0.2478, -0.2861,  0.1390,
          0.1876,  0.0212,  0.4712, -0.3445],
        [ 0.4637,  0.2263, -0.5036,  0.1986,  0.1382,  0.0208,  0.3027, -0.0782,
          0.0818, -0.5356,  0.1430,  0.2000,  0.3346, -0.3216, -0.5069,  0.3001,
          0.3305,  0.1154,  0.3756, -0.3675],
        [-0.0590,  0.2807, -0.2577,  0.3472, -0.0188,  0.2627,  0.0815, -0.3782,
         -0.1499, -0.3914,  0.1503,  0.0436,  0.0162, -0.2902, -0.3841,  0.3341,
          0.1344,  0.0214,  0.2336, -0.0951]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1470, 0.3908, 0.0000, 0.2342, 0.1481, 0.1198, 0.1688, 0.0000, 0.0590,
         0.0000, 0.1446, 0.1083, 0.1193, 0.0000, 0.0000, 0.1390, 0.1876, 0.0212,
         0.4712, 0.0000],
        [0.4637, 0.2263, 0.0000, 0.1986, 0.1382, 0.0208, 0.3027, 0.0000, 0.0818,
         0.0000, 0.1430, 0.2000, 0.3346, 0.0000, 0.00

In [32]:

seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(input_image)
print(logits)

tensor([[[2.3604e-01, 5.4428e-01, 8.0915e-01,  ..., 1.8892e-02,
          2.7146e-02, 7.7965e-01],
         [1.2414e-01, 8.4502e-01, 3.1740e-01,  ..., 4.5744e-01,
          2.0024e-01, 5.7610e-01],
         [4.7088e-01, 5.5074e-01, 7.7658e-01,  ..., 7.9615e-01,
          6.7392e-01, 3.6019e-01],
         ...,
         [6.3902e-01, 5.3565e-01, 6.2131e-01,  ..., 9.1288e-01,
          6.8873e-01, 3.0337e-01],
         [3.7434e-01, 5.3561e-01, 5.7847e-01,  ..., 3.7656e-01,
          6.8097e-02, 6.5829e-01],
         [9.0053e-03, 6.6731e-01, 4.5318e-01,  ..., 7.0869e-01,
          2.9535e-01, 2.1627e-01]],

        [[3.8783e-01, 7.1637e-01, 9.2699e-01,  ..., 2.6316e-02,
          1.0120e-01, 5.8739e-01],
         [7.9937e-01, 2.8726e-01, 7.1137e-01,  ..., 8.4997e-01,
          1.7829e-01, 6.6541e-01],
         [4.9522e-01, 4.2272e-01, 1.6936e-01,  ..., 6.9761e-01,
          4.0154e-01, 3.4533e-02],
         ...,
         [3.1499e-01, 1.0321e-01, 6.0152e-01,  ..., 9.1505e-01,
          1.174

In [35]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(softmax)
print(pred_probab)

Softmax(dim=1)
tensor([[0.0888, 0.0960, 0.0724, 0.1119, 0.0887, 0.0792, 0.0747, 0.1400, 0.1105,
         0.1378],
        [0.0962, 0.1116, 0.0816, 0.0978, 0.0955, 0.0757, 0.0855, 0.1348, 0.0935,
         0.1279],
        [0.0965, 0.0929, 0.0780, 0.1044, 0.0994, 0.0804, 0.0765, 0.1371, 0.1021,
         0.1326]], grad_fn=<SoftmaxBackward0>)
